In [1]:
import sys; sys.path.append("../../automl/")

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from src.automl.model import AutoML
from src.automl.model.metrics import RocAuc

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
[2024-11-02 14:00:47]
/home/peter/venvs/base_venv/lib/python3.10/site-packages/lightautoml/ml_algo/dl_model.py:42: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/

## Constants

In [3]:
RANDOM_SEED = 77
DATA_PATH = Path("../../data/")

## Data

In [4]:
df_train = pd.read_parquet(DATA_PATH / "train_preproc.parquet")
df_train, _ = train_test_split(df_train, train_size=100_000, random_state=RANDOM_SEED, stratify=df_train["target"])

In [5]:
ohe_cols = df_train.columns[df_train.columns.str.startswith("OneHotEncoder")].values.tolist()
oe_cols = df_train.columns[df_train.columns.str.startswith("OrdinalEncoder")].values.tolist()
te_cols = df_train.columns[df_train.columns.str.startswith("MeanTargetEncoder")].values.tolist()

In [6]:
# take ordinal encoded columns
X_train, y_train = df_train.drop(columns=["id", "target", "smpl"] + te_cols), df_train["target"]

In [7]:
X_train = X_train.sample(n=50, axis=1, random_state=RANDOM_SEED)

In [8]:
categorical_features = np.intersect1d(X_train.columns, ohe_cols + oe_cols).tolist()

## Model

In [9]:
model = AutoML("classification", RocAuc(), n_jobs=8, random_state=RANDOM_SEED, tuning_timeout=60 * 5)

In [10]:
model.fit(X_train, y_train, categorical_features=categorical_features, save_models=False)

[2024-11-02 14:00:51,974] - [   MODEL    ] - 1 out of 11. LightGBMClassification
[2024-11-02 14:00:51,978] - [   START    ] - Working with LightGBMClassification
[2024-11-02 14:00:51,981] - [   START    ] - Tuning LightGBMClassification
[2024-11-02 14:00:59,190] - [   OPTUNA   ] - Trial 0. New best score 0.7662935345019797 with parameters {'max_depth': 15, 'num_leaves': 333, 'min_data_in_leaf': 193, 'bagging_fraction': 0.5696572840513675, 'bagging_freq': 0, 'feature_fraction': 0.8728012348218951, 'lambda_l1': 3.2615093703448883, 'lambda_l2': 5.410678214759677, 'min_gain_to_split': 4.804703517360121, 'is_unbalance': True, 'num_iterations': 2}
[2024-11-02 14:01:11,310] - [   OPTUNA   ] - Trial 4. New best score 0.7775803316637012 with parameters {'max_depth': 15, 'num_leaves': 403, 'min_data_in_leaf': 68, 'bagging_fraction': 0.6547415930774714, 'bagging_freq': 0, 'feature_fraction': 0.6533028137401558, 'lambda_l1': 9.397650349673574, 'lambda_l2': 3.6704287316054973, 'min_gain_to_split': 